In [7]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [8]:
from sklearn.externals import joblib

# docbios = joblib.load('docbios.pkl')
# docnames = joblib.load('docnames.pkl')
docdict = joblib.load('docdict.pkl')

In [9]:
# for d in docnames[0:10]:
#     print d

In [10]:
# print docbios[0]

In [11]:
print len(docdict)

1899


### removing null descriptions

In [12]:
# docnames.index('Brian Anziska, MD')

# docbios = np.array(docbios)

# emptybios = []
# emptybios.append(np.where(docbios==''))
# print len(emptybios)

# emptybios = np.array(emptybios[0][0])

# print len(docnames)
# print len(docbios)

# print len(emptybios)

# print len(docnames)-len(emptybios)

# new_docnames = np.delete(docnames, emptybios)
# new_docbios = np.delete(docbios, emptybios)

# print len(docnames)
# print len(docbios)

# docnames = new_docnames
# docbios = new_docbios

# len(np.unique(docnames))

# 2505-1899

In [13]:
# from collections import defaultdict

# docdict = defaultdict(list)

# for name, bio in zip(docnames, docbios):
#     docdict[name] = bio

In [14]:
print docdict['Ludmila Davidov, MD']
print
print docdict['Igor Gavrilovic, MD']

Dr. Ludmila Davidov, MD is one of the country's most highly ranked doctors. Her specialties include psychiatry and she currently treats patients in Flushing, New York, Rego park, New York, and Whitestone, New York.  Dr. Davidov completed medical school at Tajik State Medical University Named After Abuali Ibn Sino and is licensed to see patients in New York.  Based on an in-depth analysis of Dr. Davidov's credentials, experience and network, she has been found to be among the 20% of doctors nationwide.  Dr. Davidov has been found to hold one or more active medical licenses, and successfully passed a malpractice history screening.

I am a neurologist with specialty training in neuro-oncology who sees patients at Memorial Sloan-Kettering facilities in both New York City and Basking Ridge, New Jersey. I treat patients with primary brain tumors and patients with neurological complications of systemic cancer. As a neuro-oncologist, I provide continuity of care for those patients with primary

In [15]:
len(docdict)

1899

In [16]:
#joblib.dump(docdict,'docdict.pkl')

In [17]:
2691-2051

640

### stopwords, stemming and tokenizing

In [18]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [19]:
print len(stopwords)

127


In [20]:
print stopwords[:10]

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your']


In [21]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [22]:
# here he defines a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [23]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in docdict.values():
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [24]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame'

there are 135761 items in vocab_frame


In [25]:
vocab_frame.head()

,words
dr.,dr.
ludmila,ludmila
davidov,davidov
md,md
is,is


In [26]:
print vocab_frame.head()

           words
dr.          dr.
ludmila  ludmila
davidov  davidov
md            md
is            is


### Tf-idf and document similarity

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                   use_idf=True,
                                   tokenizer=tokenize_only, ngram_range=(2,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(docdict.values()) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 6.58 s, sys: 114 ms, total: 6.69 s
Wall time: 9.49 s
(1899, 61508)


In [28]:
terms = tfidf_vectorizer.get_feature_names()

In [29]:
terms[0:10]

[u"'s adult",
 u"'s adult long-term",
 u"'s advisory",
 u"'s advisory committee",
 u"'s affairs",
 u"'s affairs medical",
 u"'s areas",
 u"'s areas of",
 u"'s best",
 u"'s best doctors"]

In [30]:
#terms_np = np.array(terms)

In [31]:
#print (np.nan in terms_np)

In [32]:
#'' in terms_np

In [33]:
print len(terms)

61508


In [34]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [35]:
print dist.shape

(1899, 1899)


### K-means clustering

In [36]:
print(tfidf_matrix.shape)

(1899, 61508)


In [95]:
from sklearn.cluster import KMeans

n = 3
km = KMeans(n_clusters=n)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 1.15 s, sys: 148 ms, total: 1.3 s
Wall time: 1.42 s


In [96]:
#from sklearn.externals import joblib

joblib.dump(km,  'doc_cluster_nostopwords.pkl')

# km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

### hacky attempt at reading level

In [97]:
sentlen = []
wordlen = []
fkgl  = []
counts = []

In [99]:
from __future__ import division
sentlen = [nltk.sent_tokenize(d) for d in docdict.values()]
wordlen = [[nltk.word_tokenize(t) for t in bio] for bio in sentlen]
for dr in wordlen:
    sentence_cnt = len(dr)
    word_cnt = 0
    letter_cnt = 0
    for sentence in dr:
        word_cnt += len(sentence)
        for word in sentence:
            letter_cnt += len(word)
    counts.append((sentence_cnt, word_cnt, letter_cnt))
    
print(counts[0])

def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except ZeroDivisionError:
        return np.nan

fkgl = [catch(lambda: 0.39 * (dr[1]/dr[0]) + 11.8 * (dr[2]/dr[1])/2.83 - 15.59) \
        for dr in counts]

(5, 119, 532)


In [100]:
fkgl.count(np.nan)

0

In [101]:
len(fkgl)

1899

In [43]:
fkgl[0]

12.332615256703388

In [44]:
max(fkgl)

47.35116607773851

In [45]:
min(fkgl)

5.826548881036516

### looking at the clusters

In [102]:
clusters[0]

0

In [103]:
print len(docdict.keys())
print len(docdict.values())
print len(fkgl)
print len(clusters)

SyntaxError: invalid syntax (<ipython-input-103-9a0048e03273>, line 1)

In [104]:
docs = { 'name': docdict.keys(), 'bio': docdict.values(), 'flesch_kincaid': fkgl, 'cluster': clusters }

In [105]:
len(vocab_frame)

135761

In [106]:
df = pd.DataFrame(docs, index = [clusters] , columns = ['name', 'flesch_kincaid', 'bio', 'cluster'])

In [107]:
df['cluster'].value_counts()

0    1335
1     389
2     175
Name: cluster, dtype: int64

In [108]:
grouped = df['flesch_kincaid'].groupby(df['cluster']) #groupby cluster for aggregation purposes

grouped.mean() #average flesch_kincaid per cluster

cluster
0    12.579876
1    12.591125
2    23.717654
Name: flesch_kincaid, dtype: float64

In [109]:
grouped.median()

cluster
0    12.085147
1    12.443783
2    23.112517
Name: flesch_kincaid, dtype: float64

In [110]:
num_clusters = len(pd.unique(df['cluster']))

In [111]:
terms[7048].split(" ")

[u'became', u'a', u'fellow']

In [56]:
#vocab_frame.ix[terms[7048].split(" ")].values.tolist()[0][0]

In [112]:
terms[33679].encode('utf-8', 'ignore')

'lapidus md'

In [58]:
terms[33679].split(" ")

[u'lapidus', u'md']

In [59]:
#vocab_frame.ix[terms[33679].split(" ")]

In [113]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        try:
            print(' %s' % terms[ind].encode('utf-8', 'ignore'), end=',')
            print()
        except AttributeError as e:
            print("AttributeError")
    print() #add whitespace
    print() #add whitespace
    
#     print("Cluster %d titles:" % i, end='')
#     for title in df.ix[i]['name'].values.tolist():
#         print(' %s,' % title, end='')
#     print() #add whitespace
#     print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: patients in,
 medical licenses,
 active medical,
 active medical licenses,
 york dr.,
 new york dr.,


Cluster 1 words: none found,
 passed a background,
 has successfully,
 has successfully passed,
 for malpractice,
 for malpractice history,


Cluster 2 words: psychiatry neurology,
 psychiatry psychiatry,
 psychiatry psychiatry neurology,
 medicine psychiatry neurology,
 medicine psychiatry,
 neurology diagnostic,






In [115]:
cluster0 = df.ix[0]
cluster1 = df.ix[1]
cluster2 = df.ix[2]

In [117]:
cluster0.head()

,name,flesch_kincaid,bio,cluster
0,"Ludmila Davidov, MD",12.332615,"Dr. Ludmila Davidov, MD is one of the country'...",0
0,"Faiq Hameedi, MD",9.976244,"Specializing in psychiatry, Dr. Faiq Hameedi, ...",0
0,"Hersha Diaz, PSYD",11.019929,"Dr. Hersha Diaz, PSYD specializes in psycholog...",0
0,"Paula Marcus, MD",10.896992,"Dr. Paula Marcus, MD specializes in psychiatry...",0
0,"Igor Gavrilovic, MD",19.305265,I am a neurologist with specialty training in ...,0


In [61]:
print(df.columns)

Index([u'name', u'flesch_kincaid', u'cluster'], dtype='object')


In [119]:
type(cluster0)

pandas.core.frame.DataFrame

In [118]:
joblib.dump(cluster0, 'doc_cluster0.pkl')
joblib.dump(cluster1, 'doc_cluster1.pkl')
joblib.dump(cluster2, 'doc_cluster2.pkl')

['doc_cluster2.pkl',
 'doc_cluster2.pkl_01.npy',
 'doc_cluster2.pkl_02.npy',
 'doc_cluster2.pkl_03.npy',
 'doc_cluster2.pkl_04.npy',
 'doc_cluster2.pkl_05.npy']

In [114]:
print(df[df['name']=='Ludmila Davidov, MD'])
print(docdict['Ludmila Davidov, MD'])
print()
print(df[df['name']=='Igor Gavrilovic, MD'])
print(docdict['Igor Gavrilovic, MD'])

                  name  flesch_kincaid  \
0  Ludmila Davidov, MD       12.332615   

                                                 bio  cluster  
0  Dr. Ludmila Davidov, MD is one of the country'...        0  
Dr. Ludmila Davidov, MD is one of the country's most highly ranked doctors. Her specialties include psychiatry and she currently treats patients in Flushing, New York, Rego park, New York, and Whitestone, New York.  Dr. Davidov completed medical school at Tajik State Medical University Named After Abuali Ibn Sino and is licensed to see patients in New York.  Based on an in-depth analysis of Dr. Davidov's credentials, experience and network, she has been found to be among the 20% of doctors nationwide.  Dr. Davidov has been found to hold one or more active medical licenses, and successfully passed a malpractice history screening.

                  name  flesch_kincaid  \
0  Igor Gavrilovic, MD       19.305265   

                                                 bio  cluster  


In [63]:
print(docdict['Sharon Lee, MD'])
print()
print(docdict['Carol Bernstein, MD'])

Dr. Sharon Lee, MD MDMPH treats patients in Somerville, Massachusetts, specializing in aerospace medicine, environmental preventive medicine, occupational medicine, preventive medical toxicology, preventive medicine, preventive sports medicine, undersea and hyperbaric medicine, dentist dental public health, and preventive medicine clinical informatics.  Dr. Lee is licensed to treat patients in Massachusetts and New York.  In addition to having active medical licenses, Dr. Lee has been found during an automated background check to be clear of any malpractice history and holds one or more active medical licenses.

Dr. Carol Bernstein, MD practices medicine at New york, New York and specializes in addiction psychiatry, child & adolescent psychiatry, clinical neurophysiology, psychiatry, psychiatry & neurology behavioral neurology & neuropsychiatry, psychiatry & neurology diagnostic neuroimaging, psychiatry & neurology forensic psychiatry, psychiatry & neurology hospice and palliative medi

In [64]:
vocab_frame.ix[terms[2].split(' ')].values.tolist()[360][0]

IndexError: list index out of range